In [3]:
# ========== Basic libraries and dependencies ==========
import numpy as np
import pandas as pd
import os
import joblib
import warnings
from datetime import datetime
import matplotlib.pyplot as plt

from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.cross_decomposition import PLSRegression
from sklearn.decomposition import PCA
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.base import clone

from pandas.tseries.offsets import BDay
from scipy.stats import f as f_dist
import statsmodels.api as sm
import matplotlib.dates as mdates
import optuna
import yfinance as yf

optuna.logging.set_verbosity(optuna.logging.WARNING)
warnings.filterwarnings('ignore')

In [4]:
# ========== Core Function Definitions ==========

def load_datasets(npz_path):
    """Load dataset from npz file"""
    data = np.load(npz_path, allow_pickle=True) 
    datasets = {}
    for key in data.files:
        datasets[key] = data[key]
    return datasets

def find_coef_step(model):
    """
    Get the coefficient step from a model, handling both Pipeline and single estimator.
    """
    if hasattr(model, 'named_steps'):
        for name, est in model.named_steps.items():
            if hasattr(est, 'coef_'):
                return name, est
            if isinstance(est, Pipeline):
                for subname, subest in est.named_steps.items():
                    if hasattr(subest, 'coef_'):
                        return f"{name}__{subname}", subest
    else:
        if hasattr(model, 'coef_'):
            return 'model', model
    
    raise ValueError("No estimator with coef_ found in model")

def annual_sharpe(rets, freq=252):
    mu = float(np.mean(rets)) * freq
    sd = float(np.std(rets, ddof=1)) * np.sqrt(freq)
    return mu / sd if sd > 0 else 0

# Load risk-free rate and calculate S&P500 Excess Sharpe
rf_file = "/Users/june/Documents/University of Manchester/Data Science/ERP/Project code/1_Data_Preprocessing/CRSP_2016_2024_top50_with_exret.csv"
rf_df = pd.read_csv(rf_file, usecols=["date", "rf"])
rf_df["date"] = pd.to_datetime(rf_df["date"])
rf_df = rf_df.drop_duplicates("date").set_index("date").sort_index()
rf_series = rf_df["rf"].astype(float)

px = yf.download("^GSPC", start="2016-01-01", end="2024-12-31")["Close"]
sp_ret = px.pct_change().dropna()
rf_align = rf_series.reindex(sp_ret.index).fillna(method="ffill")
sp_excess = sp_ret.values - rf_align.values

SR_MKT_EX = annual_sharpe(sp_excess)
print(f"[INFO] S&P500 Excess Sharpe (2016–24) = {SR_MKT_EX:.3f}")

def delta_sharpe(r2_zero, sr_base):
    sr_star = np.sqrt(sr_base**2 + r2_zero) / np.sqrt(1 - r2_zero)
    return sr_star - sr_base, sr_star

def r2_zero(y_true, y_pred):
    """
    Calculate zero-based R² (baseline is zero).
    y_true: array of true values (N,)
    y_pred: array of predicted values (N,)
    """
    rss = np.sum((y_true - y_pred)**2)  
    tss = np.sum(y_true**2)            
    return 1 - rss / tss

def calc_ic_daily(df, method='spearman'):
    """
    Calculate daily cross-sectional RankIC.
    df: must contain ['signal_date','y_true','y_pred']
    """
    ics = (df.groupby('signal_date')
             .apply(lambda g: g['y_pred'].corr(g['y_true'], method=method))
             .dropna())
    mean_ic = ics.mean()
    std_ic  = ics.std(ddof=1)
    t_ic    = mean_ic / (std_ic / np.sqrt(len(ics))) if std_ic > 0 else np.nan
    pos_ratio = (ics > 0).mean()
    return mean_ic, t_ic, pos_ratio, ics

def calc_directional_metrics(y_true, y_pred, permnos=None):
    """
    Improved version:
    - Sample-level sign prediction
    - If grouped by stock, calculate Overall, Up, Down for each stock and then average
    """
    y_true = np.asarray(y_true)
    y_pred = np.asarray(y_pred)

    if permnos is None:
        s_true = np.sign(y_true)
        s_pred = np.sign(y_pred)
        mask = s_true != 0
        s_true = s_true[mask]
        s_pred = s_pred[mask]

        overall_acc = np.mean(s_true == s_pred)

        up_mask = s_true > 0
        down_mask = s_true < 0
        up_acc = np.mean(s_true[up_mask] == s_pred[up_mask]) if np.any(up_mask) else 0
        down_acc = np.mean(s_true[down_mask] == s_pred[down_mask]) if np.any(down_mask) else 0

    else:
        df = pd.DataFrame({"permno": permnos, "yt": y_true, "yp": y_pred})
        overall_accs = []
        up_accs = []
        down_accs = []

        for _, g in df.groupby("permno"):
            s_true = np.sign(g["yt"].values)
            s_pred = np.sign(g["yp"].values)
            mask = s_true != 0
            s_true = s_true[mask]
            s_pred = s_pred[mask]
            if len(s_true) == 0:
                continue
            overall_accs.append(np.mean(s_true == s_pred))

            up_mask = s_true > 0
            down_mask = s_true < 0
            up_accs.append(np.mean(s_true[up_mask] == s_pred[up_mask]) if np.any(up_mask) else np.nan)
            down_accs.append(np.mean(s_true[down_mask] == s_pred[down_mask]) if np.any(down_mask) else np.nan)

        overall_acc = np.nanmean(overall_accs)
        up_acc = np.nanmean(up_accs)
        down_acc = np.nanmean(down_accs)

    return overall_acc, up_acc, down_acc

def regression_metrics(y_true, y_pred, k, meta=None, permnos=None):
    """
    Combined regression metrics:
    - Regression metrics
    - Pointwise directional accuracy
    - Market cap group metrics
    """
    y_true = np.asarray(y_true)
    y_pred = np.asarray(y_pred)
    n = len(y_true)

    r2 = r2_zero(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mae = mean_absolute_error(y_true, y_pred)
    mse = mean_squared_error(y_true, y_pred)

    dir_acc, up_acc, down_acc = calc_directional_metrics(y_true, y_pred, permnos)

    metrics = {
        "R²_zero": r2,
        "RMSE": rmse,
        "MAE": mae,
        "MSE": mse,
        "Directional Accuracy": dir_acc,
        "Up_Directional_Acc": up_acc,
        "Down_Directional_Acc": down_acc
    }

    if meta is not None and "MKTCAP_PERCENTILE" in meta:
        top_mask = meta["MKTCAP_PERCENTILE"] >= 0.75
        bottom_mask = meta["MKTCAP_PERCENTILE"] <= 0.25

        if np.any(top_mask):
            yt_top = y_true[top_mask]
            yp_top = y_pred[top_mask]
            perm_top = permnos[top_mask] if permnos is not None else None
            r2_top = r2_zero(yt_top, yp_top)
            rmse_top = np.sqrt(mean_squared_error(yt_top, yp_top))
            mae_top = mean_absolute_error(yt_top, yp_top)
            mse_top = mean_squared_error(yt_top, yp_top)
            dir_top, up_top, down_top = calc_directional_metrics(yt_top, yp_top, perm_top)
            metrics.update({
                "Top25_R2_zero": r2_top,
                "Top25_RMSE": rmse_top,
                "Top25_MAE": mae_top,
                "Top25_MSE": mse_top,
                "Top25_Dir_Acc": dir_top,
                "Top25_Up_Acc": up_top,
                "Top25_Down_Acc": down_top
            })

        if np.any(bottom_mask):
            yt_bot = y_true[bottom_mask]
            yp_bot = y_pred[bottom_mask]
            perm_bot = permnos[bottom_mask] if permnos is not None else None
            r2_bot = r2_zero(yt_bot, yp_bot)
            rmse_bot = np.sqrt(mean_squared_error(yt_bot, yp_bot))
            mae_bot = mean_absolute_error(yt_bot, yp_bot)
            mse_bot = mean_squared_error(yt_bot, yp_bot)
            dir_bot, up_bot, down_bot = calc_directional_metrics(yt_bot, yp_bot, perm_bot)
            metrics.update({
                "Bottom25_R2_zero": r2_bot,
                "Bottom25_RMSE": rmse_bot,
                "Bottom25_MAE": mae_bot,
                "Bottom25_MSE": mse_bot,
                "Bottom25_Dir_Acc": dir_bot,
                "Bottom25_Up_Acc": up_bot,
                "Bottom25_Down_Acc": down_bot
            })

    return metrics

def f_statistic(y_true, y_pred, k):
    """Return F statistic and corresponding p-value"""
    n   = len(y_true)
    rss = np.sum((y_true - y_pred) ** 2)
    tss = np.sum(y_true ** 2)
    r2  = 1 - rss / tss
    if (r2 <= 0) or (n <= k):
        return 0.0, 1.0
    F = (r2 / k) / ((1 - r2) / (n - k))
    p = f_dist.sf(F, k, n - k)
    return F, p

def overall_interval_metrics_method1(y_all, yhat_all, k, permnos_all=None, meta_all=None):
    """
    Method 1: Calculate metrics for the entire interval (2016-2024, all samples concatenated).
    Returns: a dict, can be directly passed to save_metrics()
    """
    base = regression_metrics(
        y_true=y_all, 
        y_pred=yhat_all, 
        k=k, 
        meta=meta_all, 
        permnos=permnos_all
    )
    F, p = f_statistic(y_all, yhat_all, k)
    base["F_stat"]     = F
    base["F_pvalue"]   = p
    base["N_obs"] = len(y_all)
    
    delta_cash, sr_star_cash = delta_sharpe(base["R²_zero"], sr_base=0)
    base["ΔSharpe_cash"]      = delta_cash
    base["Sharpe*_cash"]      = sr_star_cash

    delta_mkt , sr_star_mkt  = delta_sharpe(base["R²_zero"], sr_base=SR_MKT_EX)
    base["ΔSharpe_mkt"]       = delta_mkt
    base["Sharpe*_mkt"]       = sr_star_mkt
    
    return base

def sortino_ratio(rets, freq=252):
    """Calculate Sortino Ratio"""
    downside = rets[rets < 0]
    if len(downside) == 0:
        return np.inf
    mu = rets.mean() * freq
    sigma = np.sqrt((downside ** 2).mean()) * np.sqrt(freq)
    return mu / sigma

def cvar(rets, alpha=0.95):
    """Calculate CVaR"""
    q = np.quantile(rets, 1 - alpha)
    return rets[rets <= q].mean()

def get_model(name: str):
    """Get default model"""
    if name == "OLS":
        return LinearRegression()
    raise ValueError(f"Unexpected model: {name}")

def save_predictions(model_name, window_size, y_true, y_pred, permnos, path="predictions/"):
    os.makedirs(path, exist_ok=True)
    
    df = pd.DataFrame({
        "PERMNO": permnos,
        "y_true": y_true,
        "y_pred": y_pred
    })

    filename = f"{model_name}_w{window_size}.csv"
    df.to_csv(os.path.join(path, filename), index=False)
    print(f"[Save] {filename}")

def save_metrics(metrics_dict, name, window, path="results.csv"):
    """Save evaluation metrics"""
    row = pd.DataFrame([metrics_dict])
    row.insert(0, "Model", name)
    row.insert(1, "Window", window)

    if os.path.exists(path):
        df = pd.read_csv(path)
        df = df[~((df["Model"] == name) & (df["Window"] == window))]
        df = pd.concat([df, row], ignore_index=True)
        df.to_csv(path, index=False)
        print(f"[Update] Metrics updated for {name} w={window}")
    else:
        row.to_csv(path, index=False)
        print(f"[Create] New metrics file created with {name} w={window}")

def get_quarter_periods(start_year=2015, end_year=2024):
    """Generate quarter sequence"""
    quarters = []
    for year in range(start_year, end_year + 1):
        for q in range(1, 5):
            quarters.append((year, q))
    return quarters

def save_model_with_quarter(model, name, window, year, quarter, path="models/"):
    """Save model with quarter info"""
    os.makedirs(path, exist_ok=True)
    filename = f"{name}_w{window}_{year}Q{quarter}.joblib"
    joblib.dump(model, os.path.join(path, filename))
    print(f"Model saved: {filename}")

[*********************100%***********************]  1 of 1 completed

[INFO] S&P500 Excess Sharpe (2016–24) = 0.652


### 超参数调优

In [5]:
TUNED_MODELS = {"Ridge", "Lasso", "Enet", "PCR", "PLS"}

def tune_model_with_optuna(model_name, X, y, permnos=None, n_trials=50):
    """Hyperparameter tuning function"""
    if model_name not in TUNED_MODELS:
        print(f"Skip {model_name} - not tunable")
        return None

    tscv = TimeSeriesSplit(n_splits=3)

    def _max_nc(n_features: int) -> int:
        return max(1, min(30, n_features - 1))

    def objective(trial):
        if model_name == "Ridge":
            alpha         = trial.suggest_float("alpha", 1e-3, 10.0, log=True)
            fit_intercept = trial.suggest_categorical("fit_intercept", [True, False])
            solver        = trial.suggest_categorical("solver", ["auto", "svd", "cholesky", "lsqr"])
            base_model    = Ridge(
                alpha=alpha,
                fit_intercept=fit_intercept,
                solver=solver
            )

        elif model_name == "Lasso":
            alpha         = trial.suggest_float("alpha", 1e-4, 1, log=True)
            fit_intercept = trial.suggest_categorical("fit_intercept", [True, False])
            base_model    = Lasso(
                alpha=alpha,
                fit_intercept=fit_intercept,
                max_iter=2000
            )

        elif model_name == "Enet":
            alpha         = trial.suggest_float("alpha", 1e-4, 1.0, log=True)
            l1_ratio      = trial.suggest_float("l1_ratio", 0.1, 0.9)
            fit_intercept = trial.suggest_categorical("fit_intercept", [True, False])
            base_model    = ElasticNet(
                alpha=alpha,
                l1_ratio=l1_ratio,
                fit_intercept=fit_intercept,
                max_iter=2000
            )

        elif model_name == "PLS":
            max_c    = _max_nc(X.shape[1])
            n_comp   = trial.suggest_int("n_components", 2, max_c)
            scale    = trial.suggest_categorical("scale", [True, False])
            base_model = PLSRegression(
                n_components=n_comp,
                scale=scale
            )

        elif model_name == "PCR":
            max_c    = _max_nc(X.shape[1])
            n_comp   = trial.suggest_int("n_components", 2, max_c)
            whiten   = trial.suggest_categorical("whiten", [True, False])
            pca      = PCA(n_components=n_comp, whiten=whiten)
            base_model = make_pipeline(
                pca,
                LinearRegression()
            )

        model = base_model

        cv_mse = []
        for train_idx, val_idx in tscv.split(X):
            X_tr, X_val = X[train_idx], X[val_idx]
            y_tr, y_val = y[train_idx], y[val_idx]
            model.fit(X_tr, y_tr)
            preds = model.predict(X_val)
            cv_mse.append(mean_squared_error(y_val, preds))
        return np.mean(cv_mse)

    study = optuna.create_study(direction="minimize",
                                sampler=optuna.samplers.TPESampler(seed=42))
    study.optimize(objective, n_trials=n_trials, n_jobs=-1)

    print(f"[Hyper] {model_name}: best_MSE={study.best_value:.6f}, params={study.best_params}")

    best_trial = study.best_params
    if model_name == "Ridge":
        base_model = Ridge(alpha=best_trial["alpha"])
    elif model_name == "Lasso":
        base_model = Lasso(alpha=best_trial["alpha"], max_iter=2000)
    elif model_name == "Enet":
        base_model = ElasticNet(alpha=best_trial["alpha"],
                                l1_ratio=best_trial["l1_ratio"], max_iter=2000)
    elif model_name == "PLS":
        base_model = PLSRegression(n_components=best_trial["n_components"])
    elif model_name == "PCR":
        n_comp = best_trial.get("pca__n_components",
                               best_trial.get("n_components"))
        base_model = make_pipeline(
            PCA(n_components=n_comp),
            LinearRegression()
        )

    return base_model

### 季度重训

In [6]:

def train_models_expanding_quarterly(start_year=2015, end_year=2024, window_sizes=None, model_names=None, 
                          npz_path="/Users/june/Documents/University of Manchester/Data Science/ERP/Project code/1_Data_Preprocessing/all_window_datasets.npz"):
    """
    Quarterly Expanding Window Training:
    1. 2015Q4 (first quarter): Use 2000-2015 data for tuning and training
    2. 2015Q4: Use 2000-2015Q2 data (train) + 2015Q3 data (test from previous quarter) for training
    3. Continue to add previous quarter's test data to the training set each quarter
    """
    if window_sizes is None:
        window_sizes = [5, 21, 252, 512]
    if model_names is None:
        model_names = ["OLS", "Ridge", "Lasso", "Enet", "PLS", "PCR"]
    
    print(f"Starting Quarterly Expanding Window Training ({start_year}-{end_year})")
    
    datasets = load_datasets(npz_path)
    
    best_params_cache = {}
    quarters_to_tune = {(2015, 4), (2020, 4)}
    
    for window in window_sizes:
        print(f"Processing window size: {window}")
        
        X_train_initial = datasets[f"X_train_{window}"]
        y_train_initial = datasets[f"y_train_{window}"]
        X_test_full = datasets[f"X_test_{window}"]
        y_test_full = datasets[f"y_test_{window}"]
        
        meta_train_dict = datasets[f"meta_train_{window}"].item()
        meta_test_dict = datasets[f"meta_test_{window}"].item()
        meta_train = pd.DataFrame.from_dict(meta_train_dict)
        meta_test = pd.DataFrame.from_dict(meta_test_dict)
        meta_test['ret_date'] = pd.to_datetime(meta_test['ret_date'])
        
        X_expanding = X_train_initial.copy()
        y_expanding = y_train_initial.copy()

        for model_name in model_names:
            cache_key = f"{model_name}_w{window}"

            if model_name == "OLS":
                base_model = LinearRegression()
            elif model_name == "Ridge":
                base_model = Ridge()
            elif model_name == "Lasso":
                base_model = Lasso(max_iter=2000)
            elif model_name == "Enet":
                base_model = ElasticNet(max_iter=2000)
            elif model_name == "PLS":
                base_model = PLSRegression()
            elif model_name == "PCR":
                base_model = make_pipeline(PCA(), LinearRegression())
            else:
                base_model = LinearRegression()

            best_params_cache[cache_key] = base_model
        
        quarter_periods = get_quarter_periods(start_year, end_year)

        quarter_periods = [
        (y, q) for (y, q) in quarter_periods
        if not (y == start_year and q < 4)
        and not (y == end_year and q > 3)
        ]
        for year, quarter in quarter_periods:
            print(f"  Quarter {year}Q{quarter}: Training models with expanding data")
            
            if not (year == start_year and quarter == 1):
                if quarter == 1:
                    prev_year, prev_quarter = year - 1, 4
                else:
                    prev_year, prev_quarter = year, quarter - 1
                
                prev_quarter_mask = (
                    (meta_test['ret_date'].dt.year == prev_year) & 
                    (meta_test['ret_date'].dt.quarter == prev_quarter)
                )
                if np.any(prev_quarter_mask):
                    X_prev_quarter = X_test_full[prev_quarter_mask]
                    y_prev_quarter = y_test_full[prev_quarter_mask]
                    
                    X_expanding = np.vstack([X_expanding, X_prev_quarter])
                    y_expanding = np.hstack([y_expanding, y_prev_quarter])
                    
                    print(f"    Added {np.sum(prev_quarter_mask)} samples from {prev_year}Q{prev_quarter} to training set")
            
            for model_name in model_names:
                cache_key = f"{model_name}_w{window}"

                if ((year, quarter) in quarters_to_tune) and (model_name in TUNED_MODELS):
                    print(f"    [Tuning] {model_name} for {year}Q{quarter}")
                    tuned_model = tune_model_with_optuna(
                        model_name, X_expanding, y_expanding, n_trials=25
                    )
                    if tuned_model is None:
                        raise RuntimeError(f"{model_name} tuning failed!")

                    best_params_cache[cache_key] = tuned_model

                    model = clone(tuned_model)

                elif cache_key in best_params_cache:
                    template_model = best_params_cache[cache_key]
                    model = clone(template_model)

                else:
                    template_model = LinearRegression()
                    best_params_cache[cache_key] = template_model
                    model = clone(template_model)

                model.fit(X_expanding, y_expanding)
                step_name, coef_estimator = find_coef_step(model)
                coefs = coef_estimator.coef_
                print(f"[SANITY] {model_name} w{window} {year}Q{quarter} - first5 coefs = {np.round(coefs[:5],4)}")

                save_model_with_quarter(model, model_name, window, year, quarter)
                
            print(f"    Training data size: {len(X_expanding)} samples")
        
    print("Quarterly Expanding Window Training Completed")
    return best_params_cache


In [7]:
# ===== Portfolio Core Class =====
# ===== Transaction Cost Settings =====
TC_GRID = [0.0005, 0.001, 0.002, 0.003, 0.004]  # 5, 10, 20, 30, 40 bps
TC_TAG  = {
    0.0005: "tc5",
    0.001:  "tc10", 
    0.002:  "tc20",
    0.003:  "tc30",
    0.004:  "tc40"
}

class PortfolioBacktester:
    def __init__(self):
        self.results = {}
        
    def calc_turnover(self, w_t, r_t, w_tp1):
        """Calculate turnover using the standard formula provided by the user"""
        if w_t is None:
            return np.sum(np.abs(w_tp1))
        
        gross_ret = np.sum(w_t * r_t)
        if abs(1 + gross_ret) < 1e-8:
            return np.sum(np.abs(w_tp1))
        
        passive_weight = w_t * (1 + r_t) / (1 + gross_ret)
        turnover = np.sum(np.abs(w_tp1 - passive_weight))
        return turnover
    
    def create_portfolios_with_permno_tracking(self, signals, market_caps, permnos, top_pct=0.1, bottom_pct=0.1, weight_scheme="VW"):
        """
        Create portfolio weights based on signals, strictly tracking permno alignment.
        weight_scheme: 'VW' for value-weighted, 'EW' for equal-weighted
        """
        n_stocks = len(signals)
        top_n    = max(1, int(round(n_stocks * top_pct)))
        bottom_n = max(1, int(round(n_stocks * bottom_pct)))
        
        sorted_idx = np.argsort(signals)[::-1]
        
        top_idx = sorted_idx[:top_n]
        bottom_idx = sorted_idx[-bottom_n:]
        
        portfolio_data = {}
        
        # Long-only portfolio (Top 10%)
        long_weights = np.zeros(n_stocks)
        if len(top_idx) > 0:
            if weight_scheme == "VW":
                top_market_caps = market_caps[top_idx]
                if np.sum(top_market_caps) > 0:
                    long_weights[top_idx] = top_market_caps / np.sum(top_market_caps)
            else:
                long_weights[top_idx] = 1.0 / len(top_idx)
        
        portfolio_data['long_only'] = {
            'weights': long_weights,
            'permnos': permnos.copy(),
            'selected_permnos': permnos[top_idx] if len(top_idx) > 0 else np.array([])
        }
        
        # Short-only portfolio (Bottom 10%)
        short_weights = np.zeros(n_stocks)
        if len(bottom_idx) > 0:
            if weight_scheme == "VW":
                bottom_market_caps = market_caps[bottom_idx]
                if np.sum(bottom_market_caps) > 0:
                    short_weights[bottom_idx] = -bottom_market_caps / np.sum(bottom_market_caps)
            else:
                short_weights[bottom_idx] = -1.0 / len(bottom_idx)
        
        portfolio_data['short_only'] = {
            'weights': short_weights,
            'permnos': permnos.copy(),
            'selected_permnos': permnos[bottom_idx] if len(bottom_idx) > 0 else np.array([])
        }
        
        # Long-Short portfolio (Top long + Bottom short)
        ls_raw = long_weights + short_weights

        gross_target = 2.0
        current_gross = np.sum(np.abs(long_weights)) + np.sum(np.abs(short_weights))
        scale = gross_target / current_gross if current_gross > 1e-8 else 0.0
        ls_weights = scale * ls_raw

        ls_selected_permnos = np.concatenate([
            permnos[top_idx] if len(top_idx) > 0 else np.array([]),
            permnos[bottom_idx] if len(bottom_idx) > 0 else np.array([])
        ])

        portfolio_data['long_short'] = {
            'weights': ls_weights,
            'permnos': permnos.copy(),
            'selected_permnos': ls_selected_permnos
        }

        return portfolio_data
    
    def calculate_aligned_portfolio_return(self, portfolio_weights, portfolio_permnos, actual_returns, actual_permnos):
        """Calculate portfolio return strictly aligned by permno"""
        aligned_returns = np.zeros(len(portfolio_permnos))
        
        return_dict = dict(zip(actual_permnos, actual_returns))
        
        for i, permno in enumerate(portfolio_permnos):
            if permno in return_dict:
                aligned_returns[i] = return_dict[permno]
        
        portfolio_return = np.sum(portfolio_weights * aligned_returns)
        return portfolio_return, aligned_returns

    def calculate_metrics(self, returns, turnover_series=None):
        """Calculate portfolio metrics - only returns summary metrics, not long series"""
        returns = np.array(returns)
        
        annual_return = np.mean(returns) * 252
        annual_vol = np.std(returns, ddof=1) * np.sqrt(252)
        sharpe = annual_return / annual_vol if annual_vol > 0 else 0
        
        log_cum = np.cumsum(np.log1p(returns))
        peak_log = np.maximum.accumulate(log_cum)
        dd_log = peak_log - log_cum
        max_drawdown = 1 - np.exp(-dd_log.max()) 
        max_1d_loss = np.min(returns) 
        
        avg_turnover = np.mean(turnover_series) if turnover_series is not None else 0
        
        sortino = sortino_ratio(returns)
        cvar95  = cvar(returns, alpha=0.95)

        result = {
            'annual_return': annual_return,
            'annual_vol': annual_vol,
            'sharpe': sharpe,
            'max_drawdown': max_drawdown,
            'max_1d_loss': max_1d_loss,
            'avg_turnover': avg_turnover,
            'sortino': sortino,
            'cvar95': cvar95
        }
        
        return result

In [8]:
# ========== Main function for daily prediction and next-day rebalancing portfolio simulation ==========

def run_portfolio_simulation_daily_rebalance(start_year=2016, end_year=2024, window_sizes=None, model_names=None,
                                           npz_path="/Users/june/Documents/University of Manchester/Data Science/ERP/Project code/1_Data_Preprocessing/all_window_datasets.npz"):
    """
Portfolio simulation (daily prediction, next-day rebalancing):
    1. Load quarterly models (trained with quarterly expanding window)
    2. Daily prediction to daily signals
    3. Daily portfolio construction (T+1 rebalancing, strict permno alignment)
    4. Separate summary metrics and time series data
    """
    if window_sizes is None:
        window_sizes = [5, 21, 252, 512]
    if model_names is None:
        model_names = ["OLS", "Ridge", "Lasso", "Enet", "PLS", "PCR"]
    
    print("Starting daily rebalance portfolio backtesting simulation")
    
    backtester = PortfolioBacktester()
    datasets = load_datasets(npz_path)
    
    summary_results = []
    daily_series_data = []
    pred_rows = []
    
    WEIGHT_SCHEMES = ["VW", "EW"]
    
    for window in window_sizes:
        print(f"Processing window size: {window}")
        
        X_test = datasets[f"X_test_{window}"]
        y_test = datasets[f"y_test_{window}"]
        meta_test_dict = datasets[f"meta_test_{window}"].item()
        meta_test = pd.DataFrame.from_dict(meta_test_dict)
        
        permnos_test = meta_test["PERMNO"].values
        meta_test["signal_date"]  = pd.to_datetime(meta_test["date"])
        meta_test["ret_date"]     = pd.to_datetime(meta_test["ret_date"])
        market_caps = meta_test.get("MKTCAP", np.ones(len(permnos_test)))
        
        meta_test['date'] = pd.to_datetime(meta_test["date"])
        dates_test = meta_test['signal_date']
        
        for model_name in model_names:
            for scheme in WEIGHT_SCHEMES:
                all_y_true   = []
                all_y_pred   = []
                all_permnos  = []
                all_meta     = []
                print(f"  Model: {model_name}, Scheme: {scheme}")
                
                portfolio_daily_data = {
                    'long_only': {'returns': [], 'turnovers': [], 'dates': []},
                    'short_only': {'returns': [], 'turnovers': [], 'dates': []},
                    'long_short': {'returns': [], 'turnovers': [], 'dates': []}
                }
                
                prev_portfolio_data = {'long_only': None, 'short_only': None, 'long_short': None}
                
                signals_buf = {}
                
                for year in range(start_year, min(end_year + 1, 2025)):
                    for quarter in range(1, 5):
                        # Determine model file year and quarter (T+1 logic: use previous quarter's model to predict current quarter)
                        if quarter == 1:
                            model_file_year, model_file_quarter = year - 1, 4
                        else:
                            model_file_year, model_file_quarter = year, quarter - 1
                            
                        model_path = f"models/{model_name}_w{window}_{model_file_year}Q{model_file_quarter}.joblib"
                        
                        if not os.path.exists(model_path):
                            print(f"      Skip: Model file not found {model_path}")
                            continue
                        
                        model = joblib.load(model_path)
                        
                        quarter_mask = (
                            (dates_test.dt.year == year) & 
                            (dates_test.dt.quarter == quarter)
                        )
                        if not np.any(quarter_mask):
                            continue
                        
                        X_quarter = X_test[quarter_mask]
                        y_quarter = y_test[quarter_mask]
                        permnos_quarter = permnos_test[quarter_mask]
                        market_caps_quarter = market_caps[quarter_mask]
                        dates_quarter = dates_test[quarter_mask]
                        ret_dates_quarter = meta_test.loc[quarter_mask, 'ret_date'].values
                        
                        df_quarter = pd.DataFrame({
                            'signal_date': dates_quarter,
                            'ret_date': ret_dates_quarter,
                            'permno': permnos_quarter,
                            'market_cap': market_caps_quarter,
                            'actual_return': y_quarter,                 
                            'prediction': model.predict(X_quarter)   
                        })
                        
                        # Only collect in the first scheme to avoid duplicates
                        if scheme == 'VW':
                            df_q_save = df_quarter[['signal_date','ret_date','permno',
                                                    'actual_return','prediction','market_cap']].copy()
                            df_q_save.rename(columns={'actual_return':'y_true',
                                                      'prediction':'y_pred'}, inplace=True)
                            df_q_save['model']  = model_name
                            df_q_save['window'] = window
                            pred_rows.append(df_q_save)
                        
                        all_y_true.append(df_quarter['actual_return'].values)
                        all_y_pred.append(df_quarter['prediction'].values)
                        all_permnos.append(df_quarter['permno'].values)
                        all_meta.append(meta_test.loc[quarter_mask, :])   

                        for signal_date, sig_grp in df_quarter.groupby('signal_date'):
                            # (1) Calculate today's signals and store in buffer, do not rebalance yet
                            daily_signals = (
                                sig_grp.groupby('permno')['prediction'].mean()
                                      .to_frame('prediction')
                                      .join(sig_grp.groupby('permno')['market_cap'].mean())
                            )
                            signals_buf[signal_date] = daily_signals

                            # (2) Use previous day's signals to rebalance today
                            prev_date = signal_date - pd.tseries.offsets.BDay(1)
                            if prev_date not in signals_buf:
                                continue

                            sigs = signals_buf.pop(prev_date)

                            # (3) Use today's realized returns (ret_date == signal_date)
                            ret_grp = df_quarter[df_quarter['ret_date'] == signal_date]
                            if len(ret_grp) == 0:
                                continue

                            daily_actual_returns = (
                                ret_grp.groupby('permno')['actual_return']
                                       .mean()
                                       .reindex(sigs.index, fill_value=0)
                                       .values
                            )
                            daily_permnos = sigs.index.values

                            # (4) Generate 3 sets of weights
                            portfolios_data = backtester.create_portfolios_with_permno_tracking(
                                signals      = sigs['prediction'].values,
                                market_caps  = sigs['market_cap'].values,
                                permnos      = daily_permnos,
                                weight_scheme= scheme
                            )
                            
                            for portfolio_type in ['long_only', 'short_only', 'long_short']:
                                portfolio_info = portfolios_data[portfolio_type]
                                
                                portfolio_return, aligned_returns = backtester.calculate_aligned_portfolio_return(
                                    portfolio_weights=portfolio_info['weights'],
                                    portfolio_permnos=portfolio_info['permnos'],
                                    actual_returns=daily_actual_returns,
                                    actual_permnos=daily_permnos
                                )
                                
                                if prev_portfolio_data[portfolio_type] is not None:
                                    prev_w_ser = pd.Series(
                                        prev_portfolio_data[portfolio_type]['weights'],
                                        index=prev_portfolio_data[portfolio_type]['permnos']
                                    )
                                    cur_w_ser = pd.Series(
                                        portfolio_info['weights'],
                                        index=portfolio_info['permnos']
                                    )

                                    prev_r_ser = pd.Series(
                                        prev_portfolio_data[portfolio_type]['aligned_returns'],
                                        index=prev_portfolio_data[portfolio_type]['permnos']
                                    )

                                    aligned_prev_w = prev_w_ser.reindex(cur_w_ser.index, fill_value=0).values
                                    aligned_prev_r = prev_r_ser.reindex(cur_w_ser.index, fill_value=0).values

                                    aligned_cur_w = cur_w_ser.values

                                    turnover = backtester.calc_turnover(
                                        w_t  = aligned_prev_w,
                                        r_t  = aligned_prev_r,
                                        w_tp1= aligned_cur_w
                                    )
                                else:
                                    turnover = np.sum(np.abs(portfolio_info['weights']))
                                
                                portfolio_daily_data[portfolio_type]['returns'].append(portfolio_return)
                                portfolio_daily_data[portfolio_type]['turnovers'].append(turnover)
                                portfolio_daily_data[portfolio_type]['dates'].append(signal_date)
                                
                                prev_portfolio_data[portfolio_type] = {
                                    'weights'        : portfolio_info['weights'],
                                    'permnos'        : portfolio_info['permnos'],
                                    'aligned_returns': aligned_returns      
                                }
                
                for portfolio_type in ['long_only', 'short_only', 'long_short']:
                    portfolio_data = portfolio_daily_data[portfolio_type]
                    
                    if len(portfolio_data['returns']) > 0:
                        metrics = backtester.calculate_metrics(
                            returns=portfolio_data['returns'],
                            turnover_series=portfolio_data['turnovers']
                        )
                        
                        rets = np.array(portfolio_data['returns'])
                        tovs = np.array(portfolio_data['turnovers'])

                        for tc in TC_GRID:
                            tag = TC_TAG[tc]
                            adj = rets - tovs * tc

                            ann_ret = adj.mean() * 252
                            ann_vol = adj.std(ddof=1) * np.sqrt(252)
                            sharpe  = ann_ret / ann_vol if ann_vol > 0 else 0

                            cum_adj = np.cumprod(1 + adj)
                            mdd = ((cum_adj - np.maximum.accumulate(cum_adj)) /
                                   np.maximum.accumulate(cum_adj)).min()

                            metrics[f'{tag}_annual_return'] = ann_ret
                            metrics[f'{tag}_annual_vol']    = ann_vol
                            metrics[f'{tag}_sharpe']        = sharpe
                            metrics[f'{tag}_max_drawdown']  = mdd
                        
                        summary_results.append({
                            'scheme': scheme,
                            'model': model_name,
                            'window': window,
                            'portfolio_type': portfolio_type,
                            **metrics
                        })
                        
                        rets_arr = np.array(portfolio_data['returns'])
                        tovs_arr = np.array(portfolio_data['turnovers'])
                        cum_no_tc = np.log1p(rets_arr).cumsum()

                        tc_ret_dict = {}
                        tc_cum_dict = {}
                        for tc in TC_GRID:
                            tag = TC_TAG[tc]
                            r = rets_arr - tovs_arr * tc
                            tc_ret_dict[tag] = r
                            tc_cum_dict[tag] = np.log1p(r).cumsum()

                        for i, date in enumerate(portfolio_data['dates']):
                            row = {
                                'scheme'        : scheme,
                                'model'         : model_name,
                                'window'        : window,
                                'portfolio_type': portfolio_type,
                                'date'          : str(date),
                                'return'        : rets_arr[i],
                                'turnover'      : tovs_arr[i],
                                'cumulative'    : cum_no_tc[i],
                            }
                            for tag in TC_TAG.values():
                                row[f'{tag}_return']     = tc_ret_dict[tag][i]
                                row[f'{tag}_cumulative'] = tc_cum_dict[tag][i]

                            daily_series_data.append(row)

                if scheme == "VW" and len(all_y_true) > 0:
                    y_all    = np.concatenate(all_y_true)
                    yhat_all = np.concatenate(all_y_pred)
                    perm_all = np.concatenate(all_permnos)
                    meta_all = pd.concat(all_meta, ignore_index=True)

                    k = X_test.shape[1]

                    m1_metrics = overall_interval_metrics_method1(
                        y_all, yhat_all, k,
                        permnos_all=perm_all,
                        meta_all=meta_all
                    )

                    full_pred_df = pd.concat(pred_rows, ignore_index=True)
                    mean_ic, t_ic, pos_ic, _ = calc_ic_daily(full_pred_df, method='spearman')
                    m1_metrics['RankIC_mean']  = mean_ic
                    m1_metrics['RankIC_t']     = t_ic
                    m1_metrics['RankIC_pos%']  = pos_ic

                    save_metrics(m1_metrics, name=model_name, window=window,
                        path="portfolio_metrics.csv")

    summary_df = pd.DataFrame(summary_results)
    daily_df = pd.DataFrame(daily_series_data) if daily_series_data else pd.DataFrame()
    
    tc_columns = [c for c in summary_df.columns if c.startswith('tc')]
    summary_df[tc_columns] = summary_df[tc_columns].fillna(0.0)
    
    def save_split_by_scheme(df, base_filename):
        """Helper function to save files split by scheme"""
        if df.empty:
            print(f"Warning: DataFrame is empty, skipping save for {base_filename}")
            return None, None
            
        vw_df = df[df['scheme'] == 'VW']
        ew_df = df[df['scheme'] == 'EW']
        
        vw_filename = f"{base_filename}_VW.csv"
        ew_filename = f"{base_filename}_EW.csv"
        
        vw_df.to_csv(vw_filename, index=False)
        ew_df.to_csv(ew_filename, index=False)
        
        print(f"VW results saved to {vw_filename}")
        print(f"EW results saved to {ew_filename}")
        
        return vw_filename, ew_filename
    
    save_split_by_scheme(summary_df, "portfolio_results_daily_rebalance")
    
    if not daily_df.empty:
        save_split_by_scheme(daily_df, "portfolio_daily_series")
    
    if pred_rows:
        pred_df = pd.concat(pred_rows, ignore_index=True)
        pred_df.to_csv("predictions_daily.csv", index=False)
        print(f"Saved {len(pred_df)} prediction rows to predictions_daily.csv")
    
    print(f"Generated {len(summary_results)} portfolio summary records")
    print(f"Generated {len(daily_series_data)} daily series records")
    
    return summary_df, daily_df, backtester


In [9]:
train_models_expanding_quarterly()

Starting Quarterly Expanding Window Training (2015-2024)
Processing window size: 5
  Quarter 2015Q1: Initialize model templates without hyper-parameter tuning
  Quarter 2015Q4: Training models with expanding data
[SANITY] OLS w5 2015Q4 ── first5 coefs = [-0.0002 -0.0002 -0.0002 -0.0006 -0.0003]
Model saved: OLS_w5_2015Q4.joblib
    [Tuning] Ridge for 2015Q4
[Hyper] Ridge: best_MSE=0.000282, params={'alpha': 4.9519307168992, 'fit_intercept': True, 'solver': 'cholesky'}
[SANITY] Ridge w5 2015Q4 ── first5 coefs = [-0.0002 -0.0002 -0.0002 -0.0006 -0.0003]
Model saved: Ridge_w5_2015Q4.joblib
    [Tuning] Lasso for 2015Q4
[Hyper] Lasso: best_MSE=0.000282, params={'alpha': 0.00011370893279856273, 'fit_intercept': True}
[SANITY] Lasso w5 2015Q4 ── first5 coefs = [-0.0001 -0.0001 -0.0001 -0.0005 -0.0002]
Model saved: Lasso_w5_2015Q4.joblib
    [Tuning] Enet for 2015Q4
[Hyper] Enet: best_MSE=0.000282, params={'alpha': 0.0001032446226419321, 'l1_ratio': 0.33506316377426126, 'fit_intercept': True}

{'OLS_w5': LinearRegression(),
 'Ridge_w5': Ridge(alpha=3.2557703124535298),
 'Lasso_w5': Lasso(alpha=0.0001137467614820052, max_iter=2000),
 'Enet_w5': ElasticNet(alpha=0.00013852650492280003, l1_ratio=0.29460901083234425,
            max_iter=2000),
 'PLS_w5': PLSRegression(n_components=4),
 'PCR_w5': Pipeline(steps=[('pca', PCA(n_components=2)),
                 ('linearregression', LinearRegression())]),
 'OLS_w21': LinearRegression(),
 'Ridge_w21': Ridge(alpha=8.266891506465026),
 'Lasso_w21': Lasso(alpha=0.00047104315761308706, max_iter=2000),
 'Enet_w21': ElasticNet(alpha=0.001478425417070028, l1_ratio=0.27124005918791333,
            max_iter=2000),
 'PLS_w21': PLSRegression(n_components=3),
 'PCR_w21': Pipeline(steps=[('pca', PCA(n_components=2)),
                 ('linearregression', LinearRegression())]),
 'OLS_w252': LinearRegression(),
 'Ridge_w252': Ridge(alpha=2.135269174043857),
 'Lasso_w252': Lasso(alpha=0.14472103036427988, max_iter=2000),
 'Enet_w252': ElasticNet(alp

In [9]:
run_portfolio_simulation_daily_rebalance()

Starting Daily Rebalance Portfolio Backtesting Simulation
Processing window size: 5
  Model: OLS, Scheme: VW
[Update] Metrics updated for OLS w=5
  Model: OLS, Scheme: EW
  Model: Ridge, Scheme: VW
[Update] Metrics updated for Ridge w=5
  Model: Ridge, Scheme: EW
  Model: Lasso, Scheme: VW
[Update] Metrics updated for Lasso w=5
  Model: Lasso, Scheme: EW
  Model: Enet, Scheme: VW
[Update] Metrics updated for Enet w=5
  Model: Enet, Scheme: EW
  Model: PLS, Scheme: VW
[Update] Metrics updated for PLS w=5
  Model: PLS, Scheme: EW
  Model: PCR, Scheme: VW
[Update] Metrics updated for PCR w=5
  Model: PCR, Scheme: EW
Processing window size: 21
  Model: OLS, Scheme: VW
[Update] Metrics updated for OLS w=21
  Model: OLS, Scheme: EW
  Model: Ridge, Scheme: VW
[Update] Metrics updated for Ridge w=21
  Model: Ridge, Scheme: EW
  Model: Lasso, Scheme: VW
[Update] Metrics updated for Lasso w=21
  Model: Lasso, Scheme: EW
  Model: Enet, Scheme: VW
[Update] Metrics updated for Enet w=21
  Model: En

(    scheme model  window portfolio_type  annual_return  annual_vol    sharpe  \
 0       VW   OLS       5      long_only       0.233243    0.217784  1.070984   
 1       VW   OLS       5     short_only      -0.035127    0.209065 -0.168022   
 2       VW   OLS       5     long_short       0.198116    0.237777  0.833201   
 3       EW   OLS       5      long_only       0.197063    0.228890  0.860950   
 4       EW   OLS       5     short_only      -0.051517    0.214980 -0.239637   
 ..     ...   ...     ...            ...            ...         ...       ...   
 139     VW   PCR     512     short_only      -0.188498    0.196149 -0.960995   
 140     VW   PCR     512     long_short       0.075112    0.193944  0.387285   
 141     EW   PCR     512      long_only       0.183633    0.210584  0.872018   
 142     EW   PCR     512     short_only      -0.181233    0.204837 -0.884767   
 143     EW   PCR     512     long_short       0.002400    0.175853  0.013648   
 
      max_drawdown  max_1d

In [32]:
# ---------- Main function for 5-factor regression -----------
def run_factor_regression(port_ret, factors, use_excess=True):
    df = pd.concat([port_ret, factors], axis=1, join='inner').dropna()
    df.columns = ['ret'] + list(factors.columns)
    
    if use_excess:
        y = df['ret'].values
    else:
        y = df['ret'].values - df['rf'].values
    
    X = df[['mktrf','smb','hml','rmw','cma','umd']].values
    X = sm.add_constant(X)
    
    model = sm.OLS(y, X)
    res = model.fit()
    alpha = res.params[0]          
    resid_std = res.resid.std(ddof=1)

    ir_daily = alpha / resid_std          
    ir_annual = ir_daily * np.sqrt(252)   

    y_hat = np.asarray(res.fittedvalues)
    
    out = {
        'N_obs'            : len(y),
        'alpha_daily'      : alpha,
        'alpha_annual'     : alpha*252,      
        't_alpha'          : res.tvalues[0],
        'IR_daily'         : ir_daily,
        'IR_annual'        : ir_annual,
        'R2_zero'          : r2_zero(y, y_hat),
    }
    
    factor_names = ['MKT','SMB','HML','RMW','CMA','UMD']
    for i, fac in enumerate(factor_names, start=1):
        out[f'beta_{fac}'] = res.params[i]
        out[f't_{fac}']    = res.tvalues[i]
    
    return out

# ---------- Batch run (EW/VW, three portfolio types) ----------
def batch_factor_analysis(
    daily_df: pd.DataFrame,
    factors_path: str,
    scheme: str,
    tc_levels=(0, 5, 10, 20, 40),
    portfolio_types=('long_only','short_only','long_short'),
    model_filter=None,
    window_filter=None,
    gross_only=False,            
    out_dir='factor_IR_results',
):
    """
    Generate a CSV file containing IR results.
    gross_only=True  → only tc=0; False → all tc_levels.
    """
    import os
    os.makedirs(out_dir, exist_ok=True)

    fac = (pd.read_csv(factors_path, parse_dates=['date'])
             .set_index('date')
             .sort_index())

    sub = daily_df[daily_df['scheme'] == scheme].copy()
    if model_filter is not None:
        sub = sub[sub['model'].isin(model_filter)]
    if window_filter is not None:
        sub = sub[sub['window'].isin(window_filter)]

    tc_iter = (0,) if gross_only else tc_levels
    results = []

    for (model, win, ptype), g in sub.groupby(['model','window','portfolio_type']):
        g = g.sort_values('date').set_index(pd.to_datetime(g['date']))

        for tc in tc_iter:
            col = 'return' if tc == 0 else f'tc{tc}_return'
            if col not in g.columns:
                continue
            port_ret = g[col]
            stats = run_factor_regression(port_ret, fac, use_excess=True)
            stats.update({
                'scheme'        : scheme,
                'model'         : model,
                'window'        : win,
                'portfolio_type': ptype,
                'tc_bps'        : tc,
            })
            results.append(stats)

    df_out = pd.DataFrame(results)[[
        'scheme','model','window','portfolio_type','tc_bps','N_obs',
        'alpha_daily','alpha_annual','t_alpha',
        'IR_daily','IR_annual','R2_zero',
        'beta_MKT','t_MKT','beta_SMB','t_SMB',
        'beta_HML','t_HML','beta_RMW','t_RMW',
        'beta_CMA','t_CMA','beta_UMD','t_UMD'
    ]]

    tag = 'gross' if gross_only else 'net'
    fname = f'5_factor_analysis_{scheme}_{tag}.csv'
    df_out.to_csv(os.path.join(out_dir, fname), index=False)
    print(f'[Saved] {fname}')
    return df_out



def run_all_factor_tests(vw_csv="portfolio_daily_series_VW.csv",
                         ew_csv="portfolio_daily_series_EW.csv",
                         factor_csv="/Users/june/Documents/University of Manchester/Data Science/ERP/Project code/1_Data_Preprocessing/5_Factors_Plus_Momentum.csv",
                         save_dir="results",
                         y_is_excess=True,
                         hac_lags=5,
                         save_txt=True):
    vw_df = pd.read_csv(vw_csv)
    ew_df = pd.read_csv(ew_csv)

    vw_gross = batch_factor_analysis(
        vw_df, factor_csv, scheme='VW', gross_only=True)
    vw_net   = batch_factor_analysis(
        vw_df, factor_csv, scheme='VW', gross_only=False)

    ew_gross = batch_factor_analysis(
        ew_df, factor_csv, scheme='EW', gross_only=True)
    ew_net   = batch_factor_analysis(
        ew_df, factor_csv, scheme='EW', gross_only=False)

    return vw_gross, vw_net, ew_gross, ew_net
    

vw_gross, vw_net, ew_gross, ew_net = run_all_factor_tests()

[Saved] 5_factor_analysis_VW_gross.csv 
[Saved] 5_factor_analysis_VW_net.csv 
[Saved] 5_factor_analysis_EW_gross.csv 
[Saved] 5_factor_analysis_EW_net.csv 


In [ ]:
# === File paths ===
rf_file = "/Users/june/Documents/University of Manchester/Data Science/ERP/Project code/1_Data_Preprocessing/CRSP_2016_2024_top50_with_exret.csv"
vw_file = "portfolio_daily_series_VW.csv"
ew_file = "portfolio_daily_series_EW.csv"

# === Load risk-free rate (rf) data ===

rf_df = pd.read_csv(rf_file, usecols=["date", "rf"])
rf_df["date"] = pd.to_datetime(rf_df["date"])
rf_dict = dict(zip(rf_df["date"], rf_df["rf"]))


def adjust_returns_with_rf_grouped(file_path, output_path):
    df = pd.read_csv(file_path)
    # Parse date with flexible format
    df["date"] = pd.to_datetime(df["date"], format='mixed', dayfirst=True)

    # Find all return columns (exclude cumulative)
    return_cols = [col for col in df.columns if "return" in col and "cumul" not in col]

    # Set portfolio_type order to avoid groupby sorting issues
    order = ["long_only", "short_only", "long_short"]
    df["portfolio_type"] = pd.Categorical(df["portfolio_type"], categories=order, ordered=True)

    df_list = []
    for _, group in df.groupby(["scheme", "model", "window", "portfolio_type"], sort=False):
        group = group.sort_values("date").copy()
        for col in return_cols:
            group[col] = group.apply(lambda row: row[col] + rf_dict.get(row["date"], 0), axis=1)
            cum_col = col.replace("return", "cumulative")
            group[cum_col] = np.log1p(group[col]).cumsum()
        df_list.append(group)

    df_new = pd.concat(df_list).sort_values(["scheme", "model", "window", "portfolio_type", "date"])
    df_new.to_csv(output_path, index=False)
    print(f"Finished: {output_path}")

adjust_returns_with_rf_grouped(vw_file, "portfolio_daily_series_VW_with_rf.csv")
adjust_returns_with_rf_grouped(ew_file, "portfolio_daily_series_EW_with_rf.csv")


Finish: LM_portfolio_daily_series_VW_with_rf.csv
Finish: LM_portfolio_daily_series_EW_with_rf.csv


In [20]:
# ======== Download S&P500 (2016-2024) ========
sp500 = yf.download("^GSPC", start="2016-01-01", end="2024-12-31")
price_col = "Adj Close" if "Adj Close" in sp500.columns else "Close"
sp500["daily_return"] = sp500[price_col].pct_change().fillna(0)
# Cumulative log return (as in paper)
sp500["cum_return"] = np.cumsum(np.log1p(sp500["daily_return"]))
sp500 = sp500[["cum_return"]]
sp500.index = pd.to_datetime(sp500.index)

# ======== Configuration ========
files = [
    ("VW", "portfolio_daily_series_VW_with_rf.csv"),
    ("EW", "portfolio_daily_series_EW_with_rf.csv")
]
tc_levels = [0, 5, 10, 20, 40]      # Transaction cost (bps)
windows = [5, 21, 252, 512]         # Window size
strategies = ["long_only", "short_only", "long_short"]

output_dir = "Baseline_Portfolio"
os.makedirs(output_dir, exist_ok=True)

# Economic crisis periods (for shading)
crisis_periods = [
    (datetime(2018, 6, 1), datetime(2019, 1, 1), "US-China Trade War"),
    (datetime(2020, 2, 1), datetime(2020, 7, 1), "COVID-19"),
    (datetime(2022, 2, 1), datetime(2022, 6, 1), "Russia-Ukraine War"),
    (datetime(2023, 1, 1), datetime(2023, 4, 1), "US Bank Crisis"),
]

def plot_comparison_styled(df, scheme, tc, window):
    plt.figure(figsize=(15, 12))
    model_names = df["model"].unique()
    colors = plt.cm.tab10(np.linspace(0, 1, len(model_names)))

    offset_step = 0.02

    for i, strat in enumerate(strategies, 1):
        ax = plt.subplot(3, 1, i)

        # Plot S&P500 baseline
        plt.plot(sp500.index, sp500["cum_return"],
                 color="black", lw=2.5, label="S&P500 (Total Return)", zorder=10)

        for idx, model_name in enumerate(model_names):
            sub = df[(df["window"] == window) &
                     (df["portfolio_type"] == strat) &
                     (df["model"] == model_name)].sort_values("date")
            if sub.empty:
                continue

            # Select return column based on transaction cost
            if tc == 0:
                ret_col = "return"
            else:
                ret_col = f"tc{tc}_return"

            if ret_col not in sub.columns:
                continue

            # Cumulative log return
            log_cum = np.cumsum(np.log1p(sub[ret_col].values))

            y_shift = idx * offset_step
            plt.plot(sub["date"], log_cum + y_shift,
                     label=f"{model_name} ({strat.replace('_',' ').title()})",
                     lw=2, color=colors[idx], alpha=0.9)

        # Shade crisis periods
        for start, end, label in crisis_periods:
            ax.axvspan(start, end, color='grey', alpha=0.3)
            ax.text(start + pd.Timedelta(days=10),
                    ax.get_ylim()[1]*0.92, label, fontsize=8, color='grey')
        ax.xaxis.set_major_locator(mdates.YearLocator())
        ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y'))
        ax.set_ylabel("Cumulative log return (start = 0)")
        ax.set_title(f"{scheme} | Window={window} | Strategy={strat} | TC={tc} bps")
        ax.grid(alpha=0.3)
        plt.xticks(rotation=30)
        plt.legend(bbox_to_anchor=(1.04, 1), loc='upper left', fontsize=8)

    plt.tight_layout()
    fname = f"{scheme}_window{window}_TC{tc}_logreturn_offset.png"
    plt.savefig(os.path.join(output_dir, fname), dpi=300, bbox_inches='tight')
    plt.close()

# Main loop to generate all figures
for scheme, file_path in files:
    df = pd.read_csv(file_path)
    df["date"] = pd.to_datetime(df["date"])
    for tc in tc_levels:
        for window in windows:
            plot_comparison_styled(df, scheme, tc, window)

print(f"All figures have been generated and saved to: {output_dir}/")


[*********************100%***********************]  1 of 1 completed


All figures have been generated and saved to: Baseline_Portfolio/


In [ ]:

# Load R²_zero from portfolio_metrics.csv
metrics_df = pd.read_csv("portfolio_metrics.csv")[["Model", "Window", "R²_zero"]]
metrics_df.rename(columns={"Model": "model", "Window": "window"}, inplace=True)

# Process VW/EW files
for fname in ["portfolio_results_daily_rebalance_VW.csv", "portfolio_results_daily_rebalance_EW.csv"]:
    df = pd.read_csv(fname)

    # Merge R²_zero by model and window
    df = df.merge(metrics_df, on=["model", "window"], how="left")

    rows = []
    for _, row in df.iterrows():
        r2 = float(row["R²_zero"]) if not pd.isna(row["R²_zero"]) else 0.0
        if row["portfolio_type"] == "long_only":
            d_sr, sr_star = delta_sharpe(r2, SR_MKT_EX)
            row["ΔSharpe"]  = d_sr
            row["Sharpe*"]  = sr_star
            row["baseline"] = f"SPX_excess ({SR_MKT_EX:.2f})"
        else:
            d_sr, sr_star = delta_sharpe(r2, 0)
            row["ΔSharpe"]  = d_sr
            row["Sharpe*"]  = sr_star
            row["baseline"] = "cash (0)"
        rows.append(row)

    pd.DataFrame(rows).to_csv(fname, index=False)
    print(f"[Update] Delta Sharpe has been written to {fname}")

In [10]:
import pandas as pd
import numpy as np
from math import sqrt

PRED_PATH = "predictions_daily.csv"
METRICS_PATH = "portfolio_metrics.csv"
TREAT_CONSTANT_DAY_AS_ZERO = False
MIN_DAYS_FOR_STATS = 1

def _day_ic(g):
    if g["y_pred"].nunique(dropna=True) <= 1 or g["y_true"].nunique(dropna=True) <= 1:
        return 0.0 if TREAT_CONSTANT_DAY_AS_ZERO else np.nan
    return g["y_pred"].corr(g["y_true"], method="spearman")

def rankic_stats(df_group):
    ics = (df_group.groupby("signal_date", observed=True).apply(_day_ic).dropna())
    n = int(ics.shape[0])
    if n < MIN_DAYS_FOR_STATS:
        return pd.Series({"RankIC_mean": np.nan, "RankIC_t": np.nan, "RankIC_pos%": np.nan, "N_days": n})
    mean_ic = float(ics.mean())
    std_ic  = float(ics.std(ddof=1))
    t_ic    = mean_ic / (std_ic / np.sqrt(n)) if std_ic > 0 else np.nan
    pos_pct = float((ics > 0).mean())
    return pd.Series({"RankIC_mean": mean_ic, "RankIC_t": t_ic, "RankIC_pos%": pos_pct, "N_days": n})

# Read data and calculate RankIC
pred = pd.read_csv(PRED_PATH)
pred["signal_date"] = pd.to_datetime(pred["signal_date"], errors="coerce")
pred = pred.dropna(subset=["signal_date", "y_true", "y_pred", "model", "window"])
pred["window"] = pd.to_numeric(pred["window"], errors="coerce").astype("Int64")

rankic_df = (pred.groupby(["model", "window"], dropna=False)
                .apply(rankic_stats)
                .reset_index()
                .rename(columns={"model":"Model","window":"Window"}))

# Merge: keep new RankIC columns, add _old suffix to original metrics columns
metrics = pd.read_csv(METRICS_PATH)
metrics["Window"] = pd.to_numeric(metrics["Window"], errors="coerce").astype("Int64")

merged = metrics.merge(rankic_df, on=["Model","Window"], how="left", suffixes=("_old",""))

# Drop old columns (with _old suffix)
to_drop = [c for c in merged.columns if c.endswith("_old")]
merged = merged.drop(columns=to_drop)

# Overwrite file with updated metrics
merged.to_csv(METRICS_PATH, index=False)
print("[OK] New RankIC written to portfolio_metrics.csv (no more _x/_y columns)")


[OK] 覆盖写入新的 RankIC 到 portfolio_metrics.csv（不再有 _x/_y）
